In [13]:
from pathlib import Path
import pandas as pd
from time import time

from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound

from models import Program, Block, Workout, Workout_set, Exercise, Log_workout, Log_set

In [14]:
engine = create_engine(f"sqlite:///data/db/gym_database.db")
Session = sessionmaker(bind=engine)
session = Session()

# Algorithm

 1. Read excel (when log data is loaded, it's gonna overwrite everything by default)
 2. For each Block (=sheet), iterate through Workouts
     1. Identify Workout (using Program-Block-Workout)
     2. Log Workout data (header excel data for workout)
     3. Iterate through workout table data
         1. Identify no. of sets per Exercise line
         2. Assign correct characterization to each line
         3. Proceed with following exercise  

In [5]:
MACRO_NAME = "Macro Pisano"
LOGS_DIR = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/"

LOGS_DIR+MACRO_NAME+".xlsx"

'/mnt/c/Users/gonza/OneDrive/Gym/routines_log/Macro Pisano.xlsx'

In [3]:
(session.query(Log_set)
        .filter(Log_set.log_workout_id == 4)
        .all())

[<Log_set(id=44,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano ,set_id=1)>,
 <Log_set(id=45,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano ,set_id=2)>,
 <Log_set(id=46,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano ,set_id=3)>,
 <Log_set(id=47,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano ,set_id=4)>,
 <Log_set(id=48,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano con mancuernas,set_id=1)>,
 <Log_set(id=49,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonzalo septiembre2021,week=None,day=None,exercise=peso muerto rumano con mancuernas,set_id=2)>,
 <Log_set(id=50,date=2021-09-09,program=Macro Pisano,block=Micro 0 Gonza

In [5]:
# File path depending on machine
FILE = "/mnt/c/Users/gonza/OneDrive/Gym/routines_log/test/Macro Pisano.xlsx"
# FILE = "/mnt/d/OneDrive/Gym/routines_log/test/Macro Pisano.xlsx"

def load_log_data(log_file):

    ## 1st. Get log file name to assign to correct Program
    program_desc = Path(log_file).stem
    # print(program_desc)
    if not session.query(Program).filter(Program.program_desc == program_desc).one_or_none():
        raise KeyError(f"No record for {program_desc}!")
    
    ## 2nd. Iterate through blocks
    for block_desc, df_block in pd.read_excel(log_file, sheet_name=None, header=None).items():
        print(block_desc)
        # display(df_block)
        # Now we separate between header and body info (to log_workout and log_set, respectively)
        idx = df_block.index[df_block.isna().all(axis=1)].tolist()
        # We add the -1 to then add 1 in the loop and avoid including empty rows
        idx_mod = [-1] + idx + [len(df_block)]
        # Iterate through workouts
        for i in range(len(idx_mod)-1):
            df_wod = df_block.iloc[idx_mod[i]+1:idx_mod[i+1]]
            
            # Separate header (log_workout info)...
            df_wod_header = df_wod.loc[df_wod.iloc[:, 2:].isna().all(axis=1)].dropna(axis=1, how="all") \
                            .set_index(0).squeeze()
            # display(df_wod_header)
            # Get workout_id from Block and Program names and the order of workout in excel file
            workout_id = (session.query(Workout)
                                 .join(Block).join(Program)
                                 .filter(Block.block_desc == block_desc,
                                         Program.program_desc == program_desc)
                                 .order_by(Workout.workout_id)
                                 .all()[i].workout_id)
            # print(workout_id)

            # If log exists for workout_id, update the info
            log_workout = (session.query(Log_workout)
                                  .filter(Log_workout.workout_id == workout_id)
                                  .one_or_none())
            if log_workout:
                log_workout.date_workout_done = df_wod_header["Fecha"]
                log_workout.duration_min = df_wod_header["Duración (min)"]
                log_workout.intensity = df_wod_header["RPE general"]
                log_workout.comment_workout = df_wod_header["Comentario general"]
                # log_workout.date_reg =  
            # If not, insert the info
            else:
                log_workout = Log_workout(workout_id=workout_id, 
                                          date_workout_done=df_wod_header["Fecha"],
                                          duration_min=df_wod_header["Duración (min)"],
                                          intensity=df_wod_header["RPE general"],
                                          comment_workout=df_wod_header["Comentario general"])
                session.add(log_workout)
            # session.commit()
            
            # ... from body (log_set info)
            df_wod_exer = df_wod.loc[df_wod.iloc[:, 2:].notna().any(axis=1)]
            df_wod_exer.columns = df_wod_exer.iloc[0]
            df_wod_exer = df_wod_exer.iloc[1:]
            df_exer_done = df_wod_exer.loc[df_wod_exer[["¿Hecho?", "RPE"]].notnull().any(axis=1)]
            # display(df_exer_done)

            # Iterate through sets
            for index, row in df_exer_done.iterrows():
                # If log exists for wod_set_id, update info
                log_set = (session.query(Log_set)
                                  .filter(Log_set.workout_set_id == row["ID"])
                                  .one_or_none())
                if log_set:
                    log_set.log_workout_id = log_workout.log_workout_id
                    log_set.no_reps_done = row["Repeticiones"]
                    log_set.weight_done = row["Peso (kg)"]
                    log_set.rpe_done = row["RPE"]
                    log_set.comment_set = row["Comentarios"]

                # If not, insert info
                else:
                    log_set = Log_set(workout_set_id=row["ID"],
                                      log_workout_id=log_workout.log_workout_id,
                                      no_reps_done=row["Repeticiones"],
                                      weight_done=row["Peso (kg)"],
                                      rpe_done=row["RPE"],
                                      comment_set=row["Comentarios"])
                    session.add(log_set)
                # session.commit()
    
    session.commit()


start = time()
load_log_data(FILE)
print(f"Execution time: {time() - start:.2f} seconds")

Micro 0 Gonzalo septiembre2021


0
Fecha                 2021-09-06 00:00:00
Descripción                       pierna 
Duración (min)                         70
RPE general                             7
Comentario general                    NaN
Name: 1, dtype: object

5,ID,Ejercicio,Serie,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
6,3,sentadilla,1,10,70,NaN,NaN,8,2,x,8,NaN
7,4,sentadilla,2,10,70,NaN,NaN,8,2,x,8,NaN
8,5,sentadilla,3,10,70,NaN,NaN,8,2,x,8,NaN
9,6,sentadilla,4,10,70,NaN,NaN,8,2,x,8,NaN
10,7,peso muerto,1,5,100,NaN,NaN,5,2,x,6,NaN
11,8,peso muerto,2,5,100,NaN,NaN,5,2,x,6,NaN
12,9,peso muerto,3,5,100,NaN,NaN,5,2,x,6,NaN
13,10,peso muerto,4,5,100,NaN,NaN,5,2,x,6,NaN
14,11,extensión de cuadriceps,1,10,50,NaN,NaN,8,1,x,8,NaN
15,12,extensión de cuadriceps,2,10,50,NaN,NaN,8,1,x,8,NaN


0
Fecha                 2021-09-07 00:00:00
Descripción                         torso
Duración (min)                         70
RPE general                             7
Comentario general                    NaN
Name: 1, dtype: object

33,ID,Ejercicio,Serie,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
34,24,press de banca,1,10,60,60,NaN,7,2,x,7,NaN
35,25,press de banca,2,10,60,60,NaN,7,2,x,7,NaN
36,26,press de banca,3,10,60,60,NaN,7,2,x,7,NaN
37,27,press de banca,4,10,60,60,NaN,7,2,x,7,NaN
38,28,press plano mancuernas,1,10,25,70,NaN,7,2,x,8,NaN
39,29,press plano mancuernas,2,10,25,70,NaN,7,2,x,8,NaN
40,30,press plano mancuernas,3,10,25,70,NaN,7,2,x,8,NaN
41,31,press plano mancuernas,4,10,25,70,NaN,7,2,x,8,NaN
42,32,dominadas libres,1,8,80,70,NaN,6,2,x,6,NaN
43,33,dominadas libres,2,8,80,70,NaN,6,2,x,6,NaN


...

0
Fecha                                   2021-10-24 00:00:00
Descripción                                     pierna post
Duración (min)                                           90
RPE general                                               8
Comentario general    Orden alterado con pierna esta semana
Name: 1, dtype: object

77,ID,Ejercicio,Serie,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
78,523,"peso muerto sumo + excéntrico 3""",1,5,105,65,NaN,7,3,NaN,7,NaN
79,524,"peso muerto sumo + excéntrico 3""",2,5,105,65,NaN,7,3,NaN,8,NaN
80,525,"peso muerto sumo + excéntrico 3""",3,5,105,65,NaN,7,3,NaN,8,NaN
81,526,"peso muerto sumo + excéntrico 3""",4,5,105,65,NaN,7,3,NaN,8,NaN
82,527,"peso muerto sumo + excéntrico 3""",5,5,105,65,NaN,7,3,NaN,8,NaN
83,528,peso muerto rumano,1,6,80,80,NaN,8,2,NaN,7,NaN
84,529,peso muerto rumano,2,6,80,80,NaN,8,2,NaN,8,NaN
85,530,peso muerto rumano,3,6,80,80,NaN,8,2,NaN,8,NaN
86,531,peso muerto rumano,4,6,80,80,NaN,8,2,NaN,8,NaN
87,532,peso muerto rumano,5,6,80,80,NaN,8,2,NaN,9,NaN


0
Fecha                                   2021-10-21 00:00:00
Descripción                                          torso 
Duración (min)                                          NaN
RPE general                                               8
Comentario general    Orden alterado con pierna esta semana
Name: 1, dtype: object

114,ID,Ejercicio,Serie,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min),¿Hecho?,RPE,Comentarios
115,553,press inclinado mancuernas,1,8,28,85,NaN,8,2,x,8,No había mancuernas de 30kg
116,554,press inclinado mancuernas,2,8,28,85,NaN,8,2,x,8,No había mancuernas de 30kg
117,555,press inclinado mancuernas,3,8,28,85,NaN,8,2,x,8,No había mancuernas de 30kg
118,556,press inclinado mancuernas,1,10,22,75,NaN,8,2,x,8,NaN
119,557,press inclinado mancuernas,2,10,22,75,NaN,8,2,x,8,NaN
120,558,press inclinado mancuernas,3,10,22,75,NaN,8,2,x,8,NaN
121,559,maquina contractora de pecho,1,12,25,70,NaN,5,1,x,5,"Máquina ""floja"" había que cargarla mucho"
122,560,maquina contractora de pecho,2,12,65,70,NaN,5,1,x,5,"Máquina ""floja"" había que cargarla mucho"
123,561,maquina contractora de pecho,3,12,75,70,NaN,5,1,x,7,"Máquina ""floja"" había que cargarla mucho"
124,562,maquina contractora de pecho,4,12,70,70,NaN,5,1,x,6,"Máquina ""floja"" había que cargarla mucho"


Execution time: 1.81 seconds


In [11]:
import re

txt = "2, 2, 5.1"

# [int(s) for s in txt.split() if s.isdigit()]
# [int(s) for s in re.findall(r"\d+", txt)]
[int(s) for s in re.split(r", |,", txt) if s.isnumeric()]

txt = "60, 50"
[float(s) for s in re.split(r", |,", txt)]

[60.0, 50.0]

In [12]:
df = df_wod_exer.astype({"Series": int, 
                         "Repeticiones": int,
                         "Peso (kg)": float,
                         "% 1RM": float,
                         "RPE mín.": int,
                         "RPE máx.": int, 
                         "Descanso (min)": float,
                         "RPE": int}, 
                        errors="ignore")

for _, row in df.iterrows():
    if isinstance(row["Series"], int):
        # print(row)
        for set_id in range(1, row["Series"]+1):
            print(set_id)
            print(row.drop("Series"))
    else:
        no_sets = [int(s) for s in re.split(r", |,", row["Series"]) if s.isnumeric()]
        try:
            weights = [float(s) for s in re.split(r", |,", row["Peso (kg)"])]
        except ValueError:
            raise("Not a float!")
        
        set_id = 0
        for idx, sets in enumerate(no_sets):
            for set_ in range(1, sets+1):
                set_id += 1
                print(set_id)
                print(row.drop(["Series", "Peso (kg)"]))
                print(weights[idx])

# type(df["Series"].iloc[1])

1
44
Ejercicio                                           press inclinado
Repeticiones                                                     10
% 1RM                                                          60.0
RPE mín.                                                        NaN
RPE máx.                                                          7
Descanso (min)                                                  2.0
¿Hecho?                                                           x
RPE                                                               8
Comentarios       Muy forzado (2 últimas series con 50kg) porque...
Name: 45, dtype: object
60.0
2
44
Ejercicio                                           press inclinado
Repeticiones                                                     10
% 1RM                                                          60.0
RPE mín.                                                        NaN
RPE máx.                                                          7
Descanso 

In [13]:
pd.read_sql(
    session.query(Workout_set.workout_set_id,
                  Exercise.exercise_desc.label("Ejercicio"),
                  Workout_set.set_id.label("Serie ID"),
                  Workout_set.no_reps.label("Repeticiones"),
                  Workout_set.weight.label("Peso (kg)"),
                  Workout_set.perc_rm.label("% 1RM"),
                  Workout_set.min_rpe.label("RPE mín."),
                  Workout_set.max_rpe.label("RPE máx."),
                  Workout_set.rest_min.label("Descanso (min)"))
    # .group_by(Exercise.exercise_desc)
    .filter(Exercise.exercise_id == Workout_set.exercise_id,
            Workout_set.workout_id == 15)
    .order_by(Workout_set.workout_set_id)
    .statement,
session.bind)

,workout_set_id,Ejercicio,Serie ID,Repeticiones,Peso (kg),% 1RM,RPE mín.,RPE máx.,Descanso (min)
0,269,press inclinado,1,10,60.0,60.0,None,7,2.0
1,270,press inclinado,2,10,60.0,60.0,None,7,2.0
2,271,press inclinado,3,10,60.0,60.0,None,7,2.0
3,272,press inclinado,4,10,60.0,60.0,None,7,2.0
4,273,apertura de mancuernas en banco inclinado,1,10,12.0,70.0,None,8,1.0
5,274,apertura de mancuernas en banco inclinado,2,10,12.0,70.0,None,8,1.0
6,275,apertura de mancuernas en banco inclinado,3,10,12.0,70.0,None,8,1.0
7,276,apertura de mancuernas en banco inclinado,4,10,12.0,70.0,None,8,1.0
8,277,remo barra,1,10,60.0,60.0,None,8,2.0
9,278,remo barra,2,10,60.0,60.0,None,8,2.0
